In [1]:
!pip install datasets
import re
!pip install evaluate
!pip install sacrebleu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.2 MB/s eta 0:00:00
  

In [2]:
import os
import re
import gc
import torch
import pandas as pd
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    set_seed
)


def configure_device():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    torch.cuda.empty_cache()
    if torch.cuda.is_available():
        torch.cuda.set_per_process_memory_fraction(0.9)
        torch.backends.cudnn.benchmark = True
    return device

device = configure_device()
set_seed(42)

def load_and_preprocess_dataset(dataset_name, train_ratio=0.8):

    dataset = load_dataset(dataset_name)
    df = dataset['train'].to_pandas()
    df.rename(columns={'rm': 'Banglish', 'bn': 'Bangla'}, inplace=True)


    train_df, val_df = train_test_split(df, test_size=1 - train_ratio, random_state=42)
    return train_df, val_df

def preprocess_text(text):
    if pd.isna(text): return ""
    text = ' '.join(text.split())
    text = re.sub(r'[!@#$%^&*()_+={}\[\]:;<>?,./]', '', text)
    replacements = {
        'aa': 'a', 'ee': 'i', 'oo': 'u',
        'kh': 'k', 'sh': 's', 'th': 't',
        'dh': 'd', 'ph': 'f', 'gh': 'g',
        'ch': 'c', 'nn': 'n', 'mm': 'm'
    }
    for old, new in replacements.items():
        text = text.replace(old, new)
    text = re.sub(r'\d+', '<NUM>', text)
    return text.lower()

def preprocess_dataset(df):
    df = df.dropna()
    df['Banglish'] = df['Banglish'].apply(preprocess_text)
    df = df.drop_duplicates(subset=['Banglish']).reset_index(drop=True)

    def validate_lengths(example):
        try:
            banglish_len = len(str(example['Banglish']))
            bangla_len = len(str(example['Bangla']))
            ratio = banglish_len / bangla_len if bangla_len > 0 else float('inf')
            return 0.5 <= ratio <= 2.5
        except:
            return False

    dataset = Dataset.from_pandas(df)
    return dataset.filter(validate_lengths)

train_data, val_data = load_and_preprocess_dataset("SKNahin/bengali-transliteration-data")
train_dataset = preprocess_dataset(train_data)
val_dataset = preprocess_dataset(val_data)


tokenizer = AutoTokenizer.from_pretrained("facebook/mbart-large-50")
tokenizer.src_lang = "en_XX"
tokenizer.tgt_lang = "bn_IN"

def tokenize_function(examples):
    inputs = [str(text) for text in examples['Banglish']]
    targets = [str(text) for text in examples['Bangla']]
    model_inputs = tokenizer(
        inputs,
        max_length=128,
        padding="max_length",
        truncation=True
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=128,
            padding="max_length",
            truncation=True
        )
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=train_dataset.column_names)
val_dataset = val_dataset.map(tokenize_function, batched=True, remove_columns=val_dataset.column_names)


model = AutoModelForSeq2SeqLM.from_pretrained("facebook/mbart-large-50").to(device)

training_args = Seq2SeqTrainingArguments(
    output_dir="./banglish_translator",
    eval_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    num_train_epochs=8,
    save_total_limit=2,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=100,
    save_strategy="epoch",
    weight_decay=0.01,
    warmup_steps=300,
    fp16=True,
    report_to=[],
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)


trainer.train()
trainer.save_model("./final_model")
tokenizer.save_pretrained("./final_model")


def translate_banglish_to_bengali(text, model, tokenizer, device):
    text = preprocess_text(text)
    inputs = tokenizer(text, return_tensors="pt", max_length=128, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            forced_bos_token_id=tokenizer.lang_code_to_id["bn_IN"],
            max_length=128,
            num_beams=5,
            length_penalty=1.0,
            early_stopping=True
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


model.eval()
example_translation = translate_banglish_to_bengali("ami tomake valobashi", model, tokenizer, device)
print(example_translation)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/300 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/333k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5006 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3748 [00:00<?, ? examples/s]

...

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

Epoch,Training Loss,Validation Loss
1,9.539400,0.579122
2,0.456700,0.132232
3,0.186300,0.102287
4,0.114800,0.094896
5,0.067800,0.093307
6,0.042600,0.094918
7,0.021800,0.096620


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


আমি তোমাকে ভালোবাসি
